# I. Monte-Carlo for some explicit multiplicative free convolutions.

Recall that the Marchenko-Pastur law is the universal limit for singular values of Gaussian matrices. Following Marchenko and Pastur (1967)
$$ \frac{1}{2 \pi} \frac{\sqrt{(x-l)(r-x)}}{x} dx ,$$
where 
$$r = (1+\sqrt{c})^2$$
$$l = (1-\sqrt{c})^2$$
and $c$ being the scale parameter.

In [ ]:
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt

N=500
num_bins=20

Scenario = "Case1"

In [ ]:
# Create population spectrum
if Scenario=="Case1":
    c = 0.3
    p = int(c*N)
    r = (1+np.sqrt(c))**2 #Right end of MP
    l = (1-np.sqrt(c))**2 #Left end of MP
    population_spectrum = np.ones( p )
elif Scenario=="Case2":
    c = 0.3
    p = int(c*N)
    weights = np.array( [1, 1, 1] )
    weights = weights/np.sum( weights )
    support = np.array( [0.5, 1, 4] )

    population_cdf = np.cumsum( weights )

    population_spectrum = np.zeros( (p,) )
    block_begin = 0
    for i in range( len(weights) ):
        block_end = int( population_cdf[i]*p )
        population_spectrum[block_begin:block_end] = support[i]
        block_begin = block_end

elif Scenario=="Case3":
    c = 0.2
    p = int(c*N)
    indices = np.arange( 0, p, 1)
    toeplitz_row    = 0.3**indices
    toeplitz = scipy.linalg.toeplitz( toeplitz_row)
    
    population_spectrum, U = np.linalg.eig(toeplitz)
    population_spectrum = np.sort( population_spectrum )
else:
    print( "Please specify a scenario..." )
    raise Error()

# Histogram of population spectrum
fig, ax = plt.subplots()
n, bins, patches = ax.hist(population_spectrum, num_bins, density=True)
ax.set_xlabel('Eigenvalues')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of population spectrum for p={}'.format(p))
fig.tight_layout()
plt.xlim(0,np.max(population_spectrum)+0.5)
plt.show()

interval_max = np.max(population_spectrum)+0.5
interval       = np.linspace( 0, interval_max, 100)
population_cdf = np.zeros_like( interval )
for i in range( len(interval) ):
    t = interval[i]
    population_cdf[i] = np.count_nonzero( population_spectrum <= t )
population_cdf = population_cdf/p
plt.plot( interval, population_cdf )
plt.xlabel('Eigenvalues')
plt.ylabel('Probability')
plt.title(r'CDF of population spectrum for p={}'.format(p))
plt.xlim(0,np.max(population_spectrum)+0.5)
plt.show()

In [ ]:
from freeDeconvolution import sampling

# Sample
print( "Sampling... ")
diag = sampling.sample_wishart( p, N, population_spectrum )

# Histogram of singular values
fig, ax = plt.subplots()
n, bins, patches = ax.hist(diag, num_bins, density=True)
if Scenario=='Case1':
    y = np.sqrt( (r-bins)*(bins-l) )/(2*np.pi*bins*c) # Added extra c. I believe this is the c part of the mass, while (1-c) is a Dirac at zero
    ax.plot(bins, y, '--', linewidth=4)
ax.set_xlabel('Eigenvalues')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of singular values for p={}'.format(p))
fig.tight_layout()
plt.xlim(0,np.max(diag)+0.5)
plt.show()

# II. Cauchy-Stieljes transform: Empirical vs Theoretical

For convenience, let us compute:
$$ G(z) = \int_\mathbb{R} \frac{\mu_{MP}(dt)}{z-t}$$

The formula is
$$ G(z) = \frac{z+1-c - \sqrt{ (z-1-c)^2 - 4c} }{2z}$$
In particular
$$ M(z) = z G(z) - 1 = \frac{1}{2} ( z-1-c - \sqrt{ (z-1-c)^2 - 4c} )$$
which solves the quadratic equation:
$$ m^2 - (z-1-c)*m + c = 0$$
or equivalently
$$ m z = m(1 + c) + m^2 + c
       = (m+1)(m+c)
$$
or equivalently
$$
   s = (1+m)/(m*z)
     = 1/(m+c)
$$


In [ ]:
# BROKEN when using El Karoui's Case 1
def G_theoretical(z):
    sqrt = np.sqrt( (z-c-1)*(z-c-1)-4*c )
    positive_imag = (np.imag(sqrt)>0)
    sqrt = positive_imag*sqrt - (1-positive_imag)*sqrt
    #res = (z+c-1-sqrt)/(2*c*z)-2/(c*z)
    #return res*c
    return c*(z-c+1-sqrt)/(2*z)

from freeDeconvolution import core

mu_observed = core.DiscreteMeasure( diag, None)

z0 = np.array( complex(1.0+1.0j) )
print(z0)
print( G_theoretical(z0) )
print( mu_observed.G_empirical(z0) )

### II. 1. Zeroes of the second kind

In [ ]:
mu_observed.compute_second_kind()

zeroes_first_kind   = mu_observed.zeroes_first_kind
zeroes_second_kind  = mu_observed.zeroes_second_kind

# Plotting
plt.figure( figsize=(20,10) )
plt.scatter( np.real( zeroes_first_kind  ), np.imag( zeroes_first_kind  ), c='r', label='First kind')
plt.scatter( np.real( zeroes_second_kind ), np.imag( zeroes_second_kind ), marker='x', c='b', label='Second kind')
plt.legend()
plt.show()

### II. 2. Argument principle for locating branch points

In [ ]:
degree = len(diag)

print("Eigenvalues")
print("min: ", np.min(diag))
print("max: ", np.max(diag))

mesh_size = 10000
radius = np.max(diag)/2 + 1
center = np.max(diag)/2
interval = np.linspace(0, 2*np.pi, mesh_size)
contour = center + radius*( np.cos(interval) + np.sin(interval)*1.0j)
plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.plot( np.real(contour), np.imag(contour) )
plt.show()

def index_integrand(z):
    values = mu_observed.Markov_Krein_prime(z)/mu_observed.Markov_Krein(z)
    return values

values = index_integrand(contour)
dz = 1.0j*(contour-center)*2*np.pi/(mesh_size) 
index  = np.sum(dz*values)/(2*np.pi*1.0j)
print( "Index: ", index)
print( "Root count: ", index+2*degree)


In [ ]:
from freeDeconvolution import boxes

print( "Box segments enumeration: ")
print( boxes.box_segments_enum )

def compute_index( box, mesh_size, plot=True, color='b'):
       interval =  np.linspace( 0,1, mesh_size)
       integral = 0
       for segment in boxes.box_segments_enum:
              vector = box[ segment[1] ] - box[ segment[0] ]
              origin = box[ segment[0] ]
              s = origin + interval*vector
              #
              values = index_integrand(s)
              dz = ( s[-1]-s[0] )/mesh_size
              integral = integral + np.sum( values*dz )
              #
              if plot:
                     x = np.real(s)
                     y = np.imag(s)
                     plt.plot( x, y, c=color)
       return integral/(2*np.pi*1.0j)

In [ ]:
mesh_size = 10000
radius = 1
box = {
       'top_left'    : -1.0 + radius*1.0j,
       'bottom_right': np.max(diag) + 1 - radius*1.0j,
}
box   = boxes.extend_box(box)
index = compute_index( box, mesh_size, plot=True)
index = np.real(index+2*degree)
root_count = np.round( index )
error = index-root_count
print( "Index: ", index)
print( "Root count: ", root_count)

plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.show()

In [ ]:
def plot_box( box, mesh_size, color):
    interval =  np.linspace( 0,1, mesh_size)
    segments = []

    s = box['bottom_right'] + interval*( box['top_right'] - box['bottom_right'] )
    segments.append( s )
    s = box['top_right'] + interval*( box['top_left'] - box['top_right'] )
    segments.append( s )
    s = box['top_left'] + interval*( box['bottom_left'] - box['top_left'] )
    segments.append( s )
    s = box['bottom_left'] + interval*( box['bottom_right'] - box['bottom_left'] )
    segments.append( s )

    for s in segments:
        x = np.real(s)
        y = np.imag(s)
        plt.plot( x, y, c=color)      

    return

mesh_size = int(1e5)

plt.figure( figsize=(20,10) )

# Pass 0: Bounding box
print("Pass 0:")
radius = 1
box = {
       'top_left'    : -1.0 + radius*1.0j,
       'bottom_right': np.max(diag) + 1 - radius*1.0j,
}
box   = boxes.extend_box(box)
index = compute_index( box, mesh_size, plot=True)
index = np.real(index+2*degree)
root_count = int( np.round( index ) )
error = index-root_count
print( "Index: ", index)
print( "Root count: ", root_count)
print( "")


root_counter = 0
total_roots  = int( 0.5*root_count ) # Total number of roots in upper half plane
box['bottom_left'] = box['top_left'] # For initialization, bottom_left needs to be the previous top_left

# Loop for multiple passes and more
boxes_with_roots = []
for i in range(13):
       print(f"Pass {i+1}:")
       #print("Height: ", radius)
       radius = radius/2
       box = {
              'top_left'    : box['bottom_left'],
              'bottom_right': np.max(diag) + 1 + radius*1.0j,
       }
       box   = boxes.extend_box(box)
       index = compute_index( box, mesh_size, plot=False)
       index = np.real(index)
       root_count = int( np.round( index ) )
       root_counter = root_counter + root_count
       error = index-root_count
       print( "Index: ", index)
       print( "Root count / Total: ", root_count, '/', root_counter)
       print( "Found:", root_counter, " / ", total_roots )
       print( "")
       #
       if root_count>0:
              box['root_count'] = root_count
              boxes_with_roots.append( box )
#

# Plotting
for box in boxes_with_roots:
       color = np.random.rand(3)
       plot_box( box, mesh_size, color)
plt.scatter( np.real(diag), np.imag(diag), c='r')
plt.show()

In [ ]:
len(boxes_with_roots)

In [ ]:
def refine_boxes( coarse_boxes ):
    refined_boxes = []
    for box in coarse_boxes:
        split_boxes = boxes.split_box( box )
        #
        for refined_box in split_boxes:
            index = compute_index( refined_box, mesh_size, plot=False)
            index = np.real(index)
            root_count = int( np.round( index ) )
            if root_count>0:
                refined_box['root_count'] = root_count
                refined_boxes.append( refined_box )
    return refined_boxes

refined_boxes = boxes_with_roots
print("Pass / Number of refined boxes / Total number of roots:")
i = 1
mesh_size = int(1e5)
while True:
    refined_boxes = refine_boxes( refined_boxes )
    counts = [ box['root_count'] for box in refined_boxes]
    total_count = np.sum(counts)
    print( i, ": ", len(refined_boxes), " / ", total_count )
    i = i + 1
    #
    if total_count == len(refined_boxes):
        break
# end while
print("")

# More dichotomy refining
print("Dichotomy")
for i in range(5):
    refined_boxes = refine_boxes( refined_boxes )
    counts = [ box['root_count'] for box in refined_boxes]
    total_count = np.sum(counts)
    print( i, ": ", len(refined_boxes), " / ", total_count )


In [ ]:
# Plotting
plt.figure( figsize=(20,10) )
for box in refined_boxes:
       color = np.random.rand(3)
       plot_box( box, mesh_size, color)
plt.scatter( np.real(diag), np.imag(diag), marker='*', c='r', label=f'Eigenvalues (N={N})')
plt.title( 'Eigenvalues and estimation of branching points')
plt.legend()
plt.show()

In [ ]:
from freeDeconvolution import utils

critical_points = []
errors = []
for box in refined_boxes:
    midpoint = boxes.box_midpoint( box )
    z = utils.newton_raphson( mu_observed.Markov_Krein, mu_observed.Markov_Krein_prime, midpoint)
    critical_points.append( z )
    errors.append( abs(mu_observed.Markov_Krein_empirical(z)) )
critical_points = np.array( critical_points )
branch_points = mu_observed.M_empirical( critical_points )
errors = np.array( errors )

print( "Max error:", np.max(errors) )
sorted_indices = np.argsort(errors)
print( "Errors:   ", errors[sorted_indices] )

# Plotting critical points
plt.figure( figsize=(20,10) )
plt.scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
plt.scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
plt.scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
plt.title( 'Eigenvalues and critical points')
plt.legend()
plt.show()

# Plotting branch points
## Transport colors to uniform
branch_points_color = np.abs( np.imag( critical_points ) )
branch_points_color /= np.max( branch_points_color )
uniform_sample = np.linspace( 0, 1, len(branch_points_color))
sorted_indices = np.argsort( branch_points_color )
branch_points_color = uniform_sample[ sorted_indices ]
## Plots as usual
plt.figure( figsize=(10,20) )
plt.scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
plt.scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
plt.title( 'Branch points $m = M_\mu(z)$')
plt.legend()
plt.colorbar()
plt.show()

In [ ]:
# Saving data
with open('first_kind.npy', 'wb') as f:
    np.save(f, zeroes_first_kind)
with open('second_kind.npy', 'wb') as f:
    np.save(f, zeroes_second_kind)
with open('critical_points.npy', 'wb') as f:
    np.save(f, critical_points)
with open('branch_points.npy', 'wb') as f:
    np.save(f, branch_points)